# Using GeoJSON - Apache Sedona in PySpark
This notebook will demonstrate how you can use Apache Sedona in PySpark to perform geoanalytical queries. This development environment has been set up using version 1.6.1 of Apache Sedona.
___

## Reading GeoJSON from S3
The first code cell reads the geojson file you will be asked to process in the semester project from S3 and prints its schema. Use this code to load it into Spark and access its contents.

In [1]:
from sedona.spark import *
from pyspark.sql.functions import col
from pyspark.sql import SparkSession

# Create spark Session
spark = SparkSession.builder \
    .appName("GeoJSON read") \
    .getOrCreate()

# Create sedona context
sedona = SedonaContext.create(spark)
# Read the file from s3
geojson_path = "s3://initial-notebook-data-bucket-dblab-905418150721/project_data/LA_Census_Blocks_2020.geojson"
blocks_df = sedona.read.format("geojson") \
            .option("multiLine", "true").load(geojson_path) \
            .selectExpr("explode(features) as features") \
            .select("features.*")
# Formatting magic
flattened_df = blocks_df.select( \
                [col(f"properties.{col_name}").alias(col_name) for col_name in \
                blocks_df.schema["properties"].dataType.fieldNames()] + ["geometry"]) \
            .drop("properties") \
            .drop("type")
# Print schema
flattened_df.printSchema()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
877,application_1761923966900_0889,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- BG20: string (nullable = true)
 |-- BG20FIP_CURRENT: string (nullable = true)
 |-- BGFIP20: string (nullable = true)
 |-- CB20: string (nullable = true)
 |-- CITY: string (nullable = true)
 |-- CITYCOMM: string (nullable = true)
 |-- CITYCOMM_CURRENT: string (nullable = true)
 |-- CITY_CURRENT: string (nullable = true)
 |-- COMM: string (nullable = true)
 |-- COMM_CURRENT: string (nullable = true)
 |-- COUNTY: string (nullable = true)
 |-- CT20: string (nullable = true)
 |-- CTCB20: string (nullable = true)
 |-- FEAT_TYPE: string (nullable = true)
 |-- FIP20: string (nullable = true)
 |-- FIP_CURRENT: string (nullable = true)
 |-- HD22: long (nullable = true)
 |-- HD_NAME: string (nullable = true)
 |-- HOUSING20: long (nullable = true)
 |-- OBJECTID: long (nullable = true)
 |-- POP20: long (nullable = true)
 |-- SPA22: long (nullable = true)
 |-- SPA_NAME: string (nullable = true)
 |-- SUP21: string (nullable = true)
 |-- SUP_LABEL: string (nullable = true)
 |-- ShapeSTArea: 

## Creation of `geometry` type columns from (lat, long) coordinates using `ST_Point`
The next cell block demonstrates how you can create a `geometry` type column from a coordinates pair. Sedona can execute geoanalytics queries on `geometry` data.

In [2]:
# Create a geometry type from (Lat, Long) coordinate pairs
data1 = [(1, 34.04378747880928, -118.30008569674712), (2, 20.5937, 78.9629)]
schema1 = ["id", "latitude", "longitude"]
df1 = spark.createDataFrame(data1, schema1)
df1 = df1.withColumn("geom", ST_Point("longitude", "latitude"))
df1.show()
df1.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+-----------------+-------------------+--------------------+
| id|         latitude|          longitude|                geom|
+---+-----------------+-------------------+--------------------+
|  1|34.04378747880928|-118.30008569674712|POINT (-118.30008...|
|  2|          20.5937|            78.9629|POINT (78.9629 20...|
+---+-----------------+-------------------+--------------------+

root
 |-- id: long (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- geom: geometry (nullable = true)

## Distance calculation using `ST_DistanceSphere`
The following code implements the calculation of distance between two points, given their coordinates and taking into consideratin the spherical shape of the earth. 

In [3]:
# Calculate distance
data2 = [(12.9716, 77.5946, 20.5937, 78.9629)]
schema2 = ["latitude1", "longitude1", "latitude2", "longitude2"]
df2 = spark.createDataFrame(data2, schema2)
df2 = df2.withColumn("point1", ST_Point("longitude1", "latitude1")).withColumn("point2", ST_Point("longitude2", "latitude2"))
df2 = df2.withColumn("distance_km", ST_DistanceSphere("point1", "point2")/1000) # divide with 1000 to conver into km
df2.show()
df2.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+----------+---------+----------+--------------------+--------------------+-----------------+
|latitude1|longitude1|latitude2|longitude2|              point1|              point2|      distance_km|
+---------+----------+---------+----------+--------------------+--------------------+-----------------+
|  12.9716|   77.5946|  20.5937|   78.9629|POINT (77.5946 12...|POINT (78.9629 20...|859.9436399522059|
+---------+----------+---------+----------+--------------------+--------------------+-----------------+

root
 |-- latitude1: double (nullable = true)
 |-- longitude1: double (nullable = true)
 |-- latitude2: double (nullable = true)
 |-- longitude2: double (nullable = true)
 |-- point1: geometry (nullable = true)
 |-- point2: geometry (nullable = true)
 |-- distance_km: double (nullable = true)

## Join two DataFrames on inlcusion condition using `ST_Within`
This is how you can perform a join between two DataFrames that contain `geometry` type columns.

The join condition here is whether the `geometry` defined in `df1.geom` is contained within `flattened_df.geometry`.

In [4]:
joined_df = df1 \
    .join(flattened_df, ST_Within(df1.geom, flattened_df.geometry), "inner")
joined_df.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- id: long (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- geom: geometry (nullable = true)
 |-- BG20: string (nullable = true)
 |-- BG20FIP_CURRENT: string (nullable = true)
 |-- BGFIP20: string (nullable = true)
 |-- CB20: string (nullable = true)
 |-- CITY: string (nullable = true)
 |-- CITYCOMM: string (nullable = true)
 |-- CITYCOMM_CURRENT: string (nullable = true)
 |-- CITY_CURRENT: string (nullable = true)
 |-- COMM: string (nullable = true)
 |-- COMM_CURRENT: string (nullable = true)
 |-- COUNTY: string (nullable = true)
 |-- CT20: string (nullable = true)
 |-- CTCB20: string (nullable = true)
 |-- FEAT_TYPE: string (nullable = true)
 |-- FIP20: string (nullable = true)
 |-- FIP_CURRENT: string (nullable = true)
 |-- HD22: long (nullable = true)
 |-- HD_NAME: string (nullable = true)
 |-- HOUSING20: long (nullable = true)
 |-- OBJECTID: long (nullable = true)
 |-- POP20: long (nullable = true)
 |-- SPA22: long (nu

## Aggregate `geometry` columns using `ST_Union_Aggr`
It is possible to aggregate areas defined by the `geometry` type using the following code:

In [5]:
LA_areas = flattened_df.filter(col("CITY") == "Los Angeles") \
                .groupBy("COMM") \
                .agg(ST_Union_Aggr("geometry").alias("geometry"))
LA_areas.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- COMM: string (nullable = true)
 |-- geometry: geometry (nullable = true)

In [8]:
# Added
LA_areas.show()

An error was encountered:
Invalid status code '404' from http://ec2-3-70-202-117.eu-central-1.compute.amazonaws.com:8998/sessions/877 with error payload: {"msg":"Session '877' not found."}
